# FAISS初学者教程
---
FAISS数据库的数据存储和检索都是采用基于"Index"的方式实现的，所以对于FAISS的任何操作，都需要先构建索引<br/>
索引的作用有：
- 控制数据写入方式
- 控制相似度查找方式
- 进行数据库的存储

不同类型的索引在面对数据写入和数据相似度查找时, 运用的是不同的策略（但是写入和查找用的方法一定要相同）<br/>最简单的是方式是"Flat"线性写入和索引（也被称为“暴力索引”）, 作为初学者，先利用这一个方法实现FAISS的操作流程之后再根据拓展即可.[^1]
<br/>[官方提供的其他索引方法](https://github.com/facebookresearch/faiss/wiki/Faiss-indexes)

[^1]: 切记不要刚开始就想一口气将FAISS全部掌握

### 创建索引

In [33]:
import faiss
import numpy as np
from pandas.core.interchange.from_dataframe import primitive_column_to_ndarray

data = np.random.rand(10000, 5)  # 创建一个10000行,256列的数据
dim = 5  # 后期创建索引会反复用到维度，所以先保存
index = faiss.IndexFlatL2(dim)  # 创建指定维度的索引,并使用欧式距离计算相似度

关于相似度的计算方法也有很多种，请自行查找文献进行拓展🙂

In [ ]:
index = faiss.IndexFlatIP(dim)  # 使用向量点积计算相似度

使用工厂函数自定义需要的*存取方式*与*相似度计算方法*<br/>
参数要求：维度、存取方式、相似度计算方法

In [ ]:
index = faiss.index_factory(dim, "Flat", faiss.METRIC_L2)  # 第一个索引使用工厂函数创建
index = faiss.index_factory(dim, "Flat", faiss.METRIC_INNER_PRODUCT)  # 第二个索引使用工厂函数创建

本文之后的操作都采用第一种向量方式进行存储和读取

In [34]:
# 添加向量
index.add(data)

### 数据搜索

In [36]:
# 搜索向量
query_vectors = np.random.rand(5, 5)  # 生成一个含有5行256维的向量用于相似查询
D, I = index.search(query_vectors, k=2)  # 我们可以设置k来指定返回前k个最相似的结果
print(D, I, sep='\n')

[[0.01765398 0.0217119 ]
 [0.01314255 0.01387906]
 [0.00940344 0.0136312 ]
 [0.01189595 0.01790469]
 [0.01421544 0.0148391 ]]
[[5488 9091]
 [4587 6679]
 [7374 9057]
 [5995 7848]
 [8263 4803]]


In [37]:
# 根据索引获得相应数据
relative_data = []
for i in range(5):
    single_data = [data[I[i][0]], data[I[i][1]]]
    relative_data.append(single_data)

print(np.shape(relative_data))


(5, 2, 5)


In [38]:
# 依据缩影在原数据库中查找数据
for i in range(5):
    print(f"第{i + 1}条数据查询匹配的结果是:\n")
    for j in range(2):
        print(f"第{j + 1}/2条最相似结果:", end="")
        print(relative_data[i][j])
    print("-----------------")

第1条数据查询匹配的结果是:

第1/2条最相似结果:[0.58590784 0.8022126  0.52393972 0.29686968 0.88874731]
第2/2条最相似结果:[0.75284836 0.94124825 0.5098653  0.22167534 0.84286611]
-----------------
第2条数据查询匹配的结果是:

第1/2条最相似结果:[0.78291297 0.58216694 0.75830712 0.79187478 0.55999713]
第2/2条最相似结果:[0.65474185 0.52034147 0.66332795 0.83256785 0.70685435]
-----------------
第3条数据查询匹配的结果是:

第1/2条最相似结果:[0.99209881 0.89072181 0.88768272 0.45159288 0.8838199 ]
第2/2条最相似结果:[0.90359846 0.93549261 0.84739678 0.44447022 0.87946532]
-----------------
第4条数据查询匹配的结果是:

第1/2条最相似结果:[0.60560881 0.43308964 0.99142218 0.87073686 0.43696408]
第2/2条最相似结果:[0.66131919 0.60182707 0.99617461 0.98366362 0.43489338]
-----------------
第5条数据查询匹配的结果是:

第1/2条最相似结果:[0.4647062  0.25105355 0.19566102 0.17824151 0.40105645]
第2/2条最相似结果:[0.44028213 0.26147114 0.35241974 0.30095893 0.37640514]
-----------------


### 数据删除
如果我们希望从数据库中删除某条数据,我们是先从原data中找到数据的index, 再在向量数据库中按index进行删除;当数据存入向量数据库时, FAISS会自动对每一条数据生成一个匹配的索引（从0开始）

In [39]:
# 删除指定ID的数据
index.remove_ids(np.array([1, 2]))  # 删除两条索引
print(index.ntotal)  # 查看总数,应为10000-2=9998

9998


In [ ]:
# 如果希望删除全部的向量数据
index.reset()
print(index.ntotal)

### 保存数据库
在FAISS中, 创建的index就可以等同于"数据库", 所以保存数据库, 就是保存index

In [40]:
faiss.write_index(index, 'My_first_FAISS.faiss')

### 加载数据库

In [41]:
index = faiss.read_index('My_first_FAISS.faiss')
print("读取成功!")

读取成功!


### 自定义数据库中的数据索引
有时候我们希望索引从1开始, 或者为其他更有意义的数字时, 我们可以采用自定义索引的方式<br/>
**注意:** 传入的自定义索引长度一定要与数据长度一致, 要么全都自定义索引, 要么全都自动生成


In [44]:
index = faiss.IndexFlatIP(256)  # 创建一个256维的索引
dim = 256
# 使用IndexIDMap来包装这个索引, 使其支持自定义ID
index = faiss.IndexIDMap(index)
"""
参数1: 像向量数据库中存入的向量
参数2: 作为ID传入的numpy数组
"""
index.add_with_ids(np.random.rand(10000, 256), np.arange(1, 10001))
print(index.ntotal)

10000


In [47]:
# 尝试删除ID=0的数据,预期结果是删不掉的
index.remove_ids(np.array([0]))
print(index.ntotal)

10000


在以上设置ID的时候, 我们使用的是IndexFlatIP索引类型, 这种索引类型是不自带add_with_ids函数的, 所以我们需要先使用IndexIDMap包装一下, 再使用; 但有些索引是已经写好的, 多试试就知道了